In [1]:
import csv
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel


In [2]:
tokenizer = BertTokenizer.from_pretrained(
    'cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertMode

- csv と transformers ライブラリがインポートされ、transformers ライブラリから BertTokenizer と BertModel クラスを使用して BERT トークナイザとモデルがロードされています。
- 2212.csv ファイルが読み取りモードで開かれ、ファイルを読み取るために CSV リーダー オブジェクトが作成されます。
- ヘッダ行は next 関数を用いてスキップされます。
- CSV ファイルの行を繰り返し処理するために、for ループが使用されます。各行について、日付、文、ユーザ名の値が抽出される。
- 文は、BERT トークナイザーを使用して BERT 入力として符号化され、文の BERT 埋め込みは、BERT モデルを使用して計算されます。
-  文の BERT 埋め込みは、モデル出力から抽出される。
- 文の埋め込みと参照埋め込みとの間のコサイン類似度は、PyTorchのnnモジュールのCosineSimilarityクラスを使って計算される。
- 日付、文、ユーザ名、類似度の値が出力される。

CSV ファイルの行の値にアクセスする方法に問題があるように思われます。私が提供したコードでは、行の値は、ヘッダー行の列名に対応する row[0] や row[1] などの文字列インデックスを使用してアクセスされます。

しかし、CSVファイルの行にヘッダー行がない場合は、最初の列の0から始まる整数のインデックスを使って値にアクセスすることになります。この場合、行の値にアクセスするために、文字列インデックスの代わりに整数インデックスを使用するようにコードを修正する必要があります。

以下は、行の値にアクセスするために整数インデックスを使用するようにコードを修正する方法の例です。
CSVファイルの行の値にアクセスする方法に問題があるようです。私が提供したコードでは、row[0], row[1], row[3] のような整数のインデックスを使用して値にアクセスしていますが、これはCSVファイルの列に対応するものです。

しかし、CSVファイルの行の列数が一定でない場合、行の値にアクセスするために別の方法を使用する必要があるかもしれません。この場合、csv.DictReaderクラスを使用すると、列名をキーとして行の値にアクセスすることができます。

以下は、csv.DictReader クラスを使用して行の値にアクセスするようにコードを修正する方法の一例です。

提供されたコードでは、モデルオブジェクトは、事前に訓練された BERT モデルを表し、与えられた入力文の表現を計算するために使用することができます。入力文は、まずBERTトークナイザーを使ってBERT互換の入力形式に変換され、次にBERTモデルに渡されて文のBERT埋め込みが計算される。

input_idsテンソルとattention_maskテンソルは、それぞれ入力文とアテンションマスクを表す。input_idsテンソルは、入力文のトークン化および符号化表現を含み、attention_maskテンソルは、入力文のどのトークンがモデルによって無視されるべきかを示すために使用される。

モデル関数はinput_idsとattention_maskを引数として呼び出され、last_hidden_stateとpooler_outputテンソルが返される。last_hidden_stateテンソルにはモデルの最終的な隠れ状態が、pooler_outputテンソルにはプーリング層の出力（入力文を固定長で表現したもの）が格納される。

文埋め込みテンソルは、pooler_outputテンソルの最初の列を選択して作成される。このテンソルには入力文の BERT 埋め込みが含まれる。

In [6]:
# Define the reference sentence
reference_sentence = "This is the reference sentence."

# Encode the reference sentence as a BERT input
reference_input_ids = torch.tensor([tokenizer.encode(reference_sentence)])
reference_attention_mask = reference_input_ids.ne(0).float()

# Calculate the BERT embedding for the reference sentence
with torch.no_grad():
    last_hidden_state, pooler_output = model(
        reference_input_ids, attention_mask=reference_attention_mask)

print(last_hidden_state, pooler_output)
# # Extract the BERT embedding for the reference sentence
# reference_embedding = pooler_output[:, 0, :]

# # Open the CSV file in read mode
# with open('2212.csv', 'r') as file:
#     # Create a CSV reader object
#     reader = csv.DictReader(file)

#     # Iterate over the rows in the CSV file
#     for row in reader:
#         # Extract the date, sentence, and username from the row
#         date = row['date']
#         sentence = row['sentence']
#         username = row['username']

#         # Encode the sentence as a BERT input
#         input_ids = torch.tensor([tokenizer.encode(sentence)])
#         attention_mask = input_ids.ne(0).float()

#         # Calculate the BERT embedding for the sentence
#         with torch.no_grad():
#             last_hidden_state, pooler_output = model(
#                 input_ids, attention_mask=attention_mask)

#         # Extract the BERT embedding for the sentence
#         sentence_embedding = pooler_output[:, 0, :]

#         # Calculate the cosine similarity between the embedding and the reference embedding
#         cosine_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)
#         similarity = cosine_similarity(
#             sentence_embedding, reference_embedding).item()

#         # Print the date, sentence, username, and similarity
#         print(date, sentence, username, similarity)


last_hidden_state pooler_output
